PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.


HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.

CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

Destination - The planet the passenger will be debarking to.

Age - The age of the passenger.

VIP - Whether the passenger has paid for special VIP service during the voyage.

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

Name - The first and last names of the passenger.

Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
df1=pd.read_csv('train.csv')
dftest=pd.read_csv('test.csv')

In [ ]:
print(df1.shape,dftest.shape)

In [ ]:
dftest

In [ ]:
df1

In [ ]:
# for i in df1.columns:
#     if(df1[i].dtypes=="object"):
#         x = df1[i].mode()[0]
#         df1[i]=df1[i].fillna(x)
#     else:
#         x = df1[i].mean()
#         df1[i]=df1[i].fillna(x)

In [ ]:
df1.interpolate(method ='linear', limit_direction ='forward', inplace=True)
dftest.interpolate(method ='linear', limit_direction ='forward', inplace=True)

In [ ]:
def imp(df1,x=0):
    c=x
    df=df1
    
    x=df['PassengerId'].str.split('_')
    y=df['Cabin'].str.split('/')
    df['deck']=y.str[0]
    df['side']=y.str[2]

    # print(x.str[1])
    df['id']=x.str[1]
    df['x']=x.str[0]

    df['id']=df['id'].astype(int)
    
    le=LabelEncoder()
    df['HomePlanet']=le.fit_transform(df['HomePlanet'])
    df['Cabin']=le.fit_transform(df['Cabin'])
    df['CryoSleep']=le.fit_transform(df['CryoSleep'])
    df['Destination']=le.fit_transform(df['Destination'])
    df['VIP']=le.fit_transform(df['VIP']) #false-0
    
    df['Total bill']=df['RoomService']+df['FoodCourt']+df['ShoppingMall']+df[ 'Spa']+df['VRDeck']
    z=df['PassengerId']
    df=df.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa','VRDeck','Name','PassengerId','x'],axis=1)
    df['deck']=le.fit_transform(df['deck'])
    df['side']=le.fit_transform(df['side'])
    if c==0:
        df['Transported']=le.fit_transform(df['Transported'])
        
        
    return(df,z)

In [ ]:
df,_=imp(df1)
dft1,z=imp(dftest,1)

In [ ]:
print(df.shape,dft1.shape)

In [ ]:
df

In [ ]:
dft1

In [ ]:
df.describe()

In [ ]:
df['deck'].unique()

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
df['HomePlanet'].unique()

In [ ]:
df['id'].unique()

In [ ]:
x=df.drop(['Transported'],axis=1)
y=df['Transported']

In [ ]:
x

In [ ]:
y

In [ ]:
from collections import Counter

In [ ]:
Counter(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=101,shuffle=True,test_size=0.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)

model = abc.fit(xtrain, ytrain)


y_pred = model.predict(xtest)
len(y_pred)

In [ ]:
print(accuracy_score(y_pred,ytest))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import metrics

In [ ]:

# param_grid = {'C': [0.1, 1, 10, 100],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'gamma':['scale', 'auto'],
#               'kernel': ['linear','rbf','poly','sigmoid']} 
   
param_grid={'C': [10], 'degree': [3], 'gamma': ['scale'], 'kernel': ['linear'], 'max_iter': [-1]}
grid_svm= GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid_svm.fit(xtrain, ytrain) 

# print best parameter after tuning 
# print("best:",grid.best_params_) 
grid_svmp = grid_svm.predict(xtest) 
print("accuracy score:",accuracy_score(ytest, grid_svmp))
# print(grid.best_estimator_.get_params())
# print classification report 
# print("Classification Matrix:",metrics.classification_report(ytest, grid_svmp)) 


# [CV 4/5] END ....C=100, gamma=scale, kernel=rbf;, score=0.965 total time=  17.9s

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)

tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

from sklearn.metrics import accuracy_score
print(accuracy_score(ytrain,tr_pred))
print(accuracy_score(ytest,ts_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
tg = {"n_neighbors" : range(0,10,1)}

knc = KNeighborsClassifier()

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV (knc,tg,scoring="accuracy",cv=5)
cvmodel = cv.fit(x,y)
cvmodel.best_params_

In [ ]:

knc = KNeighborsClassifier(n_neighbors=8)
model = knc.fit(xtrain,ytrain)

tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

from sklearn.metrics import accuracy_score
print(accuracy_score(ytrain,tr_pred))
print(accuracy_score(ytest,ts_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier (random_state=20,max_depth=8,min_samples_split=88,min_samples_leaf=27)
modeltr = dtc.fit(xtrain,ytrain)

tr_pred = modeltr.predict(xtrain)
ts_pred = modeltr.predict(xtest)

from sklearn.metrics import accuracy_score
print(accuracy_score(ytrain,tr_pred))
print(accuracy_score(ytest,ts_pred))

In [ ]:
# tg = {"max_depth": range(0,50,1),
#       "n_estimators" :[100,200]
#      }
# rfc = RandomForestClassifier(random_state=20)

# from sklearn.model_selection import GridSearchCV
# cv = GridSearchCV(rfc,tg,scoring="accuracy",cv=5)
# cvmodel = cv.fit(x,y)
# cvmodel.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,random_state=20,max_depth=11)
modelr = rfc.fit(xtrain,ytrain)

tr_pred = modelr.predict(xtrain)
ts_pred = modelr.predict(xtest)

from sklearn.metrics import accuracy_score
print(accuracy_score(ytrain,tr_pred))
print(accuracy_score(ytest,ts_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(random_state=2020 , hidden_layer_sizes=(10))
mlp=MLP.fit(xtrain,ytrain)

tr_pred = mlp.predict(xtrain)
ts_pred = mlp.predict(xtest)

from sklearn.metrics import accuracy_score
print(accuracy_score(ytrain,tr_pred))
print(accuracy_score(ytest,ts_pred))

In [ ]:
dft1['deck'].info()

In [ ]:
x=dft1['Age'].mean()
print(x)

dft1['Age']=dft1['Age'].fillna(x)

In [ ]:
x=dft1['Total bill'].mean()
print(x)

dft1['Total bill']=dft1['Total bill'].fillna(x)

In [ ]:
dft1.tail()

In [ ]:
dft1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rfc = RandomForestClassifier(random_state=2 ,min_samples_split=10 , max_depth=25)
cvres = cross_val_score(rfc, x, y, cv=10)
print(cvres)
print(cvres.std())
print(cvres.mean())

In [ ]:
dft1.to_csv("tt.csv")

In [ ]:
svmpred= modeltr.predict(dft1) 

In [ ]:
Counter(svmpred)

In [ ]:
z

In [ ]:
print(len(z))
final=pd.DataFrame(z)
final['Transported']=svmpred
final

In [ ]:
final=pd.DataFrame(dftest['PassengerId'])
final['Transported']=svmpred
final

In [ ]:
final['Transported']=final['Transported'].replace({0: False, 1: True})

In [ ]:
final.to_csv('treesubmission.csv',index=False)

In [ ]:
Counter(final['Transported'])